In [6]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt

In [72]:
from mtflearn.io import load_image
from mtflearn.utils import normalize_image
from mtflearn.features import ZPs, get_characteristic_length, local_max, KeyPoints
from mtflearn.denoise import denoise_svd
from mtflearn.clustering import gmm_lbs


from stemplot.utils import  plot_pca
from stemplot import imshow, colors_from_lbs

from sklearn.manifold import TSNE as tsne
from sklearn.neighbors import NearestNeighbors

In [43]:
simu_img = np.load('croped simu image.npy')
img1 = normalize_image(simu_img, 0, 1)

In [44]:
get_characteristic_length(img1)

18

In [45]:
n_components = 64
patch_size = 64
extraction_step = 4
imgf = denoise_svd(img1, n_components, patch_size, extraction_step)

Extracting reference patches...
done in 0.23s.
Singular value decomposition...
done in 1.82s.
Reconstructing...
done in 1.02s.


In [46]:
threshold = 0.05
pts = local_max(imgf, min_distance=2, threshold=threshold)

In [47]:
size = int(18*np.sqrt(3))

In [48]:
kp1 = KeyPoints(pts, img1, size)
ps1 = kp1.extract_patches(size)

In [49]:
zps = ZPs(n_max=10, size=ps1.shape[1])
zm1 = zps.fit_transform(ps1)
X1 = zm1.data
X = np.hstack([X1])

In [50]:
plot_pca(X, 2)

In [51]:
lbs = gmm_lbs(X1, 2)

In [52]:
plot_pca(X, 2, lbs=lbs)

In [53]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.imshow(img1, cmap='gray')
ax.scatter(kp1.pts[:, 0], kp1.pts[:, 1], color=colors_from_lbs(lbs), s=10)

In [54]:
pts1 = kp1.pts[lbs != 2]
kp1 = KeyPoints(pts1, img1, size)
ps1 = kp1.extract_patches(size)

zps = ZPs(n_max=10, size=ps1.shape[1])
zm1 = zps.fit_transform(ps1)
X1 = zm1.data

X = np.hstack([X1])

In [55]:
X11 = X[lbs==0]
X22 = X[lbs==1]

In [56]:
plot_pca(X22, 2, s=3)

In [57]:
if ps1[lbs==0].mean(axis=0)[size//2, size//2] > ps1[lbs==1].mean(axis=0)[size//2, size//2]:
    lbs_selected = 1
else:
    lbs_selected = 0

print(lbs_selected)

0


In [58]:
group1_pts = kp1.pts[lbs==lbs_selected]
group1_lbs= np.zeros(X11.shape[0])

group2_pts = kp1.pts[lbs!=lbs_selected]
group2_lbs= np.zeros(X22.shape[0])
group2_lbs+=2

In [59]:
all_pts = np.vstack([group1_pts,group2_pts])
all_lbs = np.hstack([group1_lbs,group2_lbs])

In [60]:
all_pts.shape,all_lbs.shape

((7276, 2), (7276,))

In [61]:
np.save('simu pts.npy',all_pts)
np.save('simu lbs.npy',all_lbs)

### process part

In [74]:
def sort_pts(pts,p):
    vectors = pts - p
    angles = np.arctan2(vectors[:, 1], vectors[:, 0])
    sorted_indices = np.argsort(angles)
    return sorted_indices

In [62]:
original_pts = np.load('simu pts.npy')
original_lbs = np.load('simu lbs.npy')
img = np.load('croped simu image.npy')

In [63]:
kp = KeyPoints(original_pts, img, size)
original_ps = kp.extract_patches(size)
zps = ZPs(n_max=12, size=original_ps.shape[1])
moments = zps.fit_transform(original_ps)

In [64]:
mask1 = original_lbs==0

In [65]:
mask1.shape,original_pts.shape,original_lbs.shape

((7276,), (7276, 2), (7276,))

In [66]:
pts = original_pts[mask1]
ps = original_ps[mask1]
Se2_lbs = original_lbs[mask1]

In [70]:
pts1 = pts[Se2_lbs == 0]#Se2 positions
X11 = tsne_X1[Se2_lbs == 0]
tsne_res = tsne(n_components=2).fit_transform(X11)

In [71]:
tsne_res.shape,pts1.shape

((3640, 2), (3640, 2))

In [73]:
nn = NearestNeighbors(n_neighbors=2)
nn.fit(pts1)

distances, indices = nn.kneighbors(pts1)
nearest_distances = distances[:, 1]
average_distance = np.mean(nearest_distances)
radius = int(average_distance*1.3)
print("Radius of further points nearest neighbor:", radius)

Radius of further points nearest neighbor: 22


In [75]:
select_pts1 = []
select_theta1 = []
angles = np.arctan2(tsne_res[:, 1], tsne_res[:, 0])
nn = NearestNeighbors(radius=radius)
nn.fit(pts1)
neighbor_limit = 7
for i, p in enumerate(pts1):
    distances, indices = nn.radius_neighbors([p])

    if i in indices[0]:
        indices = indices[0][indices[0] != i]
        distances = distances[0][distances[0] != i]

    if len(indices) >= neighbor_limit - 1:
        select_pts = pts1[indices]
        select_angles = angles[indices]
        sort_idx = sort_pts(select_pts, p)
        sort_angles = select_angles[sort_idx]
        diffs = np.diff(sort_angles)
        result = np.append(diffs, sort_angles[-1] - sort_angles[0])
        select_pts1.append(p)
        select_theta1.append(result)
select_pts1 = np.array(select_pts1)
select_theta1 = np.array(select_theta1)

In [76]:
mean_theta1 = np.mean(select_theta1,axis = 1)

In [77]:
mean_theta = mean_theta1
all_pts = select_pts1

In [78]:
norm_theta = mean_theta-mean_theta.min()
norm_theta = norm_theta/norm_theta.max()

In [83]:
new_cmap = plt.get_cmap('coolwarm')(norm_theta)
plt.style.use('default')

In [84]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
cmap = plt.get_cmap('coolwarm')
# norm = Normalize(vmin=norm_theta.min(), vmax=norm_theta.max())
norm = Normalize(vmin=-np.pi, vmax=np.pi)
fig,ax = plt.subplots(1,1,figsize=(7.2,7.2))
ax.imshow(img,cmap = 'gray')
ax.scatter(all_pts[:,0],all_pts[:,1],c = new_cmap,s = 7)
ax.axis('off')
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# 使用 make_axes_locatable 创建一个与当前轴相关的分区
divider = make_axes_locatable(ax)
# 在右侧添加一个新的轴，用于放置 colorbar
cax = divider.append_axes("right", size="5%", pad=0.05)

# 添加颜色条到新的轴上，并且设置其高度与原图相同
cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
ticks = [-np.pi,-2,-1, 0,1,2, np.pi]  # 定义ticks的位置
tick_labels = [r'$-\pi$',r'$-2$' ,r'$-1$' ,r'$0$',r'$1$' ,r'$2$' , r'$\pi$']  # 定义ticks的标签，这里使用 LaTeX 格式

cbar.set_ticks(ticks)
cbar.set_ticklabels(tick_labels)

# 如果你想要更加精确地控制colorbar的大小，可以调整size参数
# cax = divider.append_axes("right", size="100%", pad=0.05) # 这会使得colorbar宽度等于主图宽度
plt.savefig('simu delta theta coolwarm cmap.png',dpi = 700,transparent = True)